# Applied Machine Learning PA4

## Exercise Question

In [1]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import Normalizer
from sklearn.pipeline import make_pipeline
from sklearn.feature_selection import SelectKBest
from sklearn.model_selection import GridSearchCV
from sklearn.base import BaseEstimator
from sklearn.pipeline import Pipeline
import numpy as np
import time

In [2]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import Perceptron
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.pipeline import make_pipeline

X1 = [{'city':'Gothenburg', 'month':'July'},
      {'city':'Gothenburg', 'month':'December'},
      {'city':'Paris', 'month':'July'},
      {'city':'Paris', 'month':'December'}]
Y1 = ['rain', 'rain', 'sun', 'rain']

X2 = [{'city':'Sydney', 'month':'July'},
      {'city':'Sydney', 'month':'December'},
      {'city':'Paris', 'month':'July'},
      {'city':'Paris', 'month':'December'}]
Y2 = ['rain', 'sun', 'sun', 'rain']

classifier1 = make_pipeline(DictVectorizer(), Perceptron(max_iter=10))
classifier1.fit(X1, Y1)
guesses1 = classifier1.predict(X1)
print(accuracy_score(Y1, guesses1))

classifier2 = make_pipeline(DictVectorizer(), Perceptron(max_iter=10))
#classifier2 = make_pipeline(DictVectorizer(), LinearSVC())
classifier2.fit(X2, Y2)
guesses2 = classifier2.predict(X2)
print(accuracy_score(Y2, guesses2))

1.0
0.5


reasons: 

- Very Small dataset

- Overfitting 

- Generalization problem

There are some problems which cause dramatic decrease when the training set changes. All these problems are related to each other. First of all, the training sets are quite small. Training and learning from the small dataset make the model not generalize well on new data. For example, the original dataset has only two different cities and the second dataset has a new city, Sydney and the model struggles to predict this city's classification. Memorizing the training data means over fitting occurs. Instead of learning the underlying patterns in the dataset, due to lack of data, the model memorizes the dataset. 

## Introduction

In [3]:

"""This file shows a couple of implementations of the perceptron learning
algorithm. It is based on the code from Lecture 3, but using the slightly
more compact perceptron formulation that we saw in Lecture 6.

There are two versions: Perceptron, which uses normal NumPy vectors and
matrices, and SparsePerceptron, which uses sparse vectors and matrices.
The latter may be faster when we have high-dimensional feature representations
with a lot of zeros, such as when we are using a "bag of words" representation
of documents.
"""

import numpy as np
from sklearn.base import BaseEstimator

class LinearClassifier(BaseEstimator):
    """
    General class for binary linear classifiers. Implements the predict
    function, which is the same for all binary linear classifiers. There are
    also two utility functions.
    """

    def decision_function(self, X):
        """
        Computes the decision function for the inputs X. The inputs are assumed to be
        stored in a matrix, where each row contains the features for one
        instance.
        """
        return X.dot(self.w)

    def predict(self, X):
        """
        Predicts the outputs for the inputs X. The inputs are assumed to be
        stored in a matrix, where each row contains the features for one
        instance.
        """

        # First compute the output scores
        scores = self.decision_function(X)

        # Select the positive or negative class label, depending on whether
        # the score was positive or negative.
        out = np.select([scores >= 0.0, scores < 0.0],
                        [self.positive_class,
                         self.negative_class])
        return out

    def find_classes(self, Y):
        """
        Finds the set of output classes in the output part Y of the training set.
        If there are exactly two classes, one of them is associated to positive
        classifier scores, the other one to negative scores. If the number of
        classes is not 2, an error is raised.
        """
        classes = sorted(set(Y))
        if len(classes) != 2:
            raise Exception("this does not seem to be a 2-class problem")
        self.positive_class = classes[1]
        self.negative_class = classes[0]

    def encode_outputs(self, Y):
        """
        A helper function that converts all outputs to +1 or -1.
        """
        return np.array([1 if y == self.positive_class else -1 for y in Y])


class Perceptron(LinearClassifier):
    """
    A straightforward implementation of the perceptron learning algorithm.
    """

    def __init__(self, n_iter=20):
        """
        The constructor can optionally take a parameter n_iter specifying how
        many times we want to iterate through the training set.
        """
        self.n_iter = n_iter

    def fit(self, X, Y):
        """
        Train a linear classifier using the perceptron learning algorithm.
        """

        # First determine which output class will be associated with positive
        # and negative scores, respectively.
        self.find_classes(Y)

        # Convert all outputs to +1 (for the positive class) or -1 (negative).
        Ye = self.encode_outputs(Y)

        # If necessary, convert the sparse matrix returned by a vectorizer
        # into a normal NumPy matrix.
        if not isinstance(X, np.ndarray):
            X = X.toarray()

        # Initialize the weight vector to all zeros.
        n_features = X.shape[1]
        self.w = np.zeros(n_features)

        # Perceptron algorithm:
        for i in range(self.n_iter):
            for x, y in zip(X, Ye):

                # Compute the output score for this instance.
                score = x.dot(self.w)

                # If there was an error, update the weights.
                if y*score <= 0:
                    self.w += y*x


##### The following part is for the optional task.

### Sparse and dense vectors don't collaborate very well in NumPy/SciPy.
### Here are two utility functions that help us carry out some vector
### operations that we'll need.

def add_sparse_to_dense(x, w, factor):
    """
    Adds a sparse vector x, scaled by some factor, to a dense vector.
    This can be seen as the equivalent of w += factor * x when x is a dense
    vector.
    """
    w[x.indices] += factor * x.data

def sparse_dense_dot(x, w):
    """
    Computes the dot product between a sparse vector x and a dense vector w.
    """
    return np.dot(w[x.indices], x.data)


class SparsePerceptron(LinearClassifier):
    """
    A straightforward implementation of the perceptron learning algorithm,
    assuming that the input feature matrix X is sparse.
    """

    def __init__(self, n_iter=20):
        """
        The constructor can optionally take a parameter n_iter specifying how
        many times we want to iterate through the training set.
        """
        self.n_iter = n_iter

    def fit(self, X, Y):
        """
        Train a linear classifier using the perceptron learning algorithm.

        Note that this will only work if X is a sparse matrix, such as the
        output of a scikit-learn vectorizer.
        """
        self.find_classes(Y)

        # First determine which output class will be associated with positive
        # and negative scores, respectively.
        Ye = self.encode_outputs(Y)

        # Initialize the weight vector to all zeros.
        self.w = np.zeros(X.shape[1])

        # Iteration through sparse matrices can be a bit slow, so we first
        # prepare this list to speed up iteration.
        XY = list(zip(X, Ye))

        for i in range(self.n_iter):
            for x, y in XY:

                # Compute the output score for this instance.
                # (This corresponds to score = x.dot(self.w) above.)
                score = sparse_dense_dot(x, self.w)

                # If there was an error, update the weights.
                if y*score <= 0:
                    # (This corresponds to self.w += y*x above.)
                    add_sparse_to_dense(x, self.w, y)





In [4]:

import time
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import Normalizer
from sklearn.pipeline import make_pipeline
from sklearn.feature_selection import SelectKBest
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# This function reads the corpus, returns a list of documents, and a list
# of their corresponding polarity labels. 
def read_data(corpus_file):
    X = []
    Y = []
    with open(corpus_file, encoding='utf-8') as f:
        for line in f:
            _, y, _, x = line.split(maxsplit=3)
            X.append(x.strip())
            Y.append(y)
    return X, Y


if __name__ == '__main__':
    
    # Read all the documents.
    X, Y = read_data('all_sentiment_shuffled.txt')
    
    # Split into training and test parts.
    Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=0.2,
                                                    random_state=0)

    # Set up the preprocessing steps and the classifier.
    pipeline = make_pipeline(
        TfidfVectorizer(),
        SelectKBest(k=1000),
        Normalizer(),

        # NB that this is our Perceptron, not sklearn.linear_model.Perceptron
        Perceptron()  
    )

    # Train the classifier.     
    t0 = time.time()
    pipeline.fit(Xtrain, Ytrain)
    t1 = time.time()
    print('Training time: {:.2f} sec.'.format(t1-t0))

    # Evaluate on the test set.
    Yguess = pipeline.predict(Xtest)
    print('Accuracy: {:.4f}.'.format(accuracy_score(Ytest, Yguess)))




Training time: 1.19 sec.
Accuracy: 0.7919.


## Implementing the SVC

The Pegasos (Primal Estimated sub-Gradient Solver) algorithm  is a sub -gradient descent algorithm for optimizing the objective function. The objective is to minimize the loss and maximize the margin between the decision boundary and the data points. The data points that lie closest to the line are called the support vectors. In the implementation, the python class Supportvm is implemented as the Pegasos algorithm. Initially the weight vector is initialized to all zeros. The algorithm begins by iterating over the samples and chooses a set of samples randomly. The weight is updated based on the values, if the dot product value is less than  1 then the weight is updated using the regularized function and loss function, if the sample is classified correctly then the weight is updated using the regularized objective function. 

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_selection import SelectKBest
from sklearn.model_selection import GridSearchCV
from sklearn.base import BaseEstimator
from sklearn.pipeline import Pipeline
import numpy as np
import time

In [6]:
def read_data(corpus_file):
    X = []
    Y = []
    with open(corpus_file, encoding='utf-8') as f:
        for line in f:
            _, y, _, x = line.split(maxsplit=3)
            X.append(x.strip())
            Y.append(y)
    return X, Y


In [7]:
if __name__ == '__main__':

    X, Y = read_data('all_sentiment_shuffled.txt')
    Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=0.2,random_state=0)

In [8]:
class LinearClassifier(BaseEstimator):
    
    def decision_function(self, X):
        
        return X.dot(self.w)

    def predict(self, X):
        scores = self.decision_function(X)
        out = np.select([scores >= 0.0, scores < 0.0],
                        [self.positive_class,
                         self.negative_class])
        return out

    def find_classes(self, Y):
        
        classes = sorted(set(Y))
        if len(classes) != 2:
            raise Exception("this does not seem to be a 2-class problem")
        self.positive_class = classes[1]
        self.negative_class = classes[0]    

    def encode_outputs(self, Y):
        
        return np.array([1 if y == self.positive_class else -1 for y in Y])

In [9]:
class Supportvc(LinearClassifier):
    """Implementation of pegasos svc  """
    def __init__(self, lambda_value =1, n_iter = 25):
        #self.step_size = step_size
        self.lambda_value = lambda_value
        self.n_iter = n_iter
        self.w = None
    

    def fit(self, X, Y):
       
        self.find_classes(Y)
        Ye = self.encode_outputs(Y)
        X = X.toarray()
        n_features = X.shape[1]
        n_samples = X.shape[0]
        self.w = np.zeros(n_features)
  
        for t in range(self.n_iter):
            i = np.random.choice(n_samples,1)[0]
            x=X[i]
            y = Ye[i]
            eta = 1 / (self.lambda_value * (t+1))
            if y * np.dot(self.w,x ) < 1:
                self.w  = (1 - eta*self.lambda_value) * self.w + eta * y * x
            else:
                self.w  = (1 - eta*self.lambda_value) * self.w  
        return  self.w 

The pipeline class is used to streamline the process of vectorization, feature selection and model training.
The Countvectorizer, SelectKbest  and the model svm_classifier is trained using the pegasos algorithm.
The key hyperparameters of the algorithm n_iter and lambda_value is tuned to optimize the model such 
that the performance and accuracy of the model can be increased.
The model has resulted in 0.80 accuracy  show the model predicts 80 % of the data correctly.


In [29]:
svc_classifier = Pipeline( [('vectorizer', CountVectorizer(preprocessor = lambda x: x)),
    ('sk', SelectKBest(k=5000)),
    ('svm', Supportvc(n_iter=len(Xtrain)*10, lambda_value=0.015))] )
t0 = time.time()
svc_classifier.fit(Xtrain, Ytrain)
t1 = time.time()
print(' Time:', t1-t0, 'seconds.')
Yvalue = svc_classifier.predict(Xtest)
print('accuracy:', accuracy_score(Ytest, Yvalue))


 Time: 14.426332712173462 seconds.
accuracy: 0.8166177087704574



The Grid search CV is used to find the best hyper parameters for the models ,the parameter defined  are set with a range of hyper parameter values to evaluate the model accuracy .The output of the grid search show the best hyper parameters for the model and accuracy.


In [28]:

parameter = { 'sk__k': [1000,5000, 10000], 
    'svm__lambda_value': [0.1,0.015, 0.02],
    'svm__n_iter': [len(Xtrain)*10]}
grid_search = GridSearchCV(svc_classifier, parameter, cv=5,n_jobs=-1,  scoring='accuracy')
grid_search.fit(Xtrain, Ytrain)
print('Best parameters:', grid_search.best_params_)
print('Validation accuracy:', grid_search.best_score_)

## Implementing the Logistic Regression

In [18]:
class LRPegasos(LinearClassifier):
    """Implementation of LR with SGD with PEGASOS Algorithm"""
 
    def __init__(self, n_iter=1000, lambda1=0.001):
        self.n_iter = n_iter
        self.lambda1 = lambda1
        self.w = None
 
    def fit(self, X, Y,):
       
        self.find_classes(Y)
        # convert all output values to +1 or -1
        Yn = self.encode_outputs(Y) #As usual, the outputs (in the list Y) are coded as +1 for positive examples and -1 for negative examples.
        X = X.toarray()
        m, n_features = X.shape[0], X.shape[1]
        self.w = np.zeros( n_features )
        for t in range(self.n_iter):
            eta = 1. / (self.lambda1*(t+1)) #The number (Greek letter eta) is the step length or learning rate in gradient descent
            j = np.random.choice(m, 1)[0] # pick a fixed number of randomly selected training pairs.
            x, y = X[j], Yn[j]
            score = x.dot(self.w) 
            # Log Loss          
            self.w = ((1-eta*self.lambda1)*self.w) + (eta *y/(1+np.exp(y*score))*x)
           
        return self.w


It is almost the same as the SVC except for updating weights. Here, there is no condition for updating weight. The loss function differs. This is log loss and svc uses hinge loss.  This part "((1-eta*self.lambda1)*self.w)" is the same as svc and the other part is the gradient of the log loss

In [26]:
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

classifier = Pipeline( [('vectorizer', CountVectorizer(preprocessor = lambda x: x)),('fs', SelectKBest(k=5000)),('cls', LRPegasos(n_iter=len(Xtrain)*10, lambda1=0.0015))] )
t0 = time.time()
classifier.fit(Xtrain, Ytrain)
t1 = time.time()
print(' Time:', t1-t0, 'seconds.')
pred2 = classifier.predict(Xtest)
accuracy2 = accuracy_score(Ytest, pred2)
print(accuracy2)


/tmp/ipykernel_540/1630960913.py:22: RuntimeWarning: overflow encountered in exp
  self.w = ((1-eta*self.lambda1)*self.w) + (eta *y/(1+np.exp(y*score))*x)
 Time: 15.775731325149536 seconds.
0.8124213176668066


Compared to SVC, LR has almost the same accuracy and training time performance. When we tried with the different vectorizer such as TF-IDF, we get the same results. When we increased the number of iterations len(Xtrain)*100, the accuracy rose slightly by 1% but the training time went up incredibly. 

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=c247be64-9f47-45f6-9108-f49f6e841fe2' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>